# Environment Preparation

Run using Colab. Setup Runtime GPU before starting

In [0]:
#!git clone --quiet https://github.com/ThomasHenckel/CarND-Capstone/Traffic_Light_Detection.git tfd
!git clone --quiet https://github.com/ilopezfr/Traffic_Light_Detection.git tfd

!cp -r tfd/config config

!git clone --quiet https://github.com/tensorflow/models.git tf_models
  
!cp -r tf_models/research/object_detection object_detection/

!cp -r tf_models/research/slim slim/

!cp tfd/model_main.py object_detection/model_main.py

!rm -rf tfd

!rm -rf tf_models

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

!git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI

!make

%cd /content

!cp -r cocoapi/PythonAPI/pycocotools pycocotools/

!rm -rf cocoapi

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content:/content/slim

# Testing Installation

In [2]:
!python object_detection/builders/model_builder_test.py


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

.W0303 19:26:10.518628 140633413740416 deprecation.py:323] From /content/object_detection/anchor_generators/grid_anchor_generator.py:59: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
....................s
----------------------------------------------------------------------
Ran 22 tests in 0.128s

OK (skipped=1)


# Download Dataset

The ids are the training and evaluation tf record files.

In [0]:
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1orq0y-8GtfOWl1tBko03rSZT7b3sVfBf' #'1S1KDLCS2ZVhJtb-cFUtpWTs5U2sR0oMT'  (backup)
eval_file_id = '18nLlxkdJtwfbOaFvpdLhJXrknfzwNNKw'  #'1pwOAFH0CUnXAykS91XsCBI846HJiauqw' (backup)

data_dir = 'data'

if not os.path.isdir(data_dir):
  os.makedirs(data_dir)

downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'mixed_train.record'))

downloaded = drive.CreateFile({'id': eval_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'mixed_eval.record'))

# Download Pre-Trained Models

In [0]:
import urllib.request
import tarfile

MODEL_NAME =  'ssd_mobilenet_v1_coco_2018_01_28'  #'ssd_mobilenet_v1_coco_11_06_2017' use this in case the other is incompatible w/ TF 1.3
MODEL_FILE = MODEL_NAME + '.tar.gz'
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/' + MODEL_FILE

MODELS_DIR = 'models'

if not os.path.isdir(MODELS_DIR):
  os.makedirs(MODELS_DIR)

if not os.path.exists(MODEL_FILE):
  urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
os.rename(MODEL_NAME, os.path.join(MODELS_DIR, MODEL_NAME))

# Train the model 

In [5]:
FINE_TUNED_DIR = os.path.join(MODELS_DIR, 'fine_tuned')

if not os.path.isdir(FINE_TUNED_DIR):
  os.makedirs(FINE_TUNED_DIR)

!python object_detection/model_main.py \
        --pipeline_config_path=config/ssd_mobilenet_v1.config \
        --model_dir=models/fine_tuned/ssd_mobilenet \


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu False
INFO:tensorflow:Using config: {'_model_dir': 'models/fine_tuned/ssd_mobilenet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_

In [8]:
# check which version of TF do I have:
!python -c 'import tensorflow as tf; print(tf.__version__)'
!pip list | grep tensorflow

1.13.1
mesh-tensorflow          0.0.5                
tensorflow               1.13.1               
tensorflow-estimator     1.13.0               
tensorflow-hub           0.2.0                
tensorflow-metadata      0.12.1               
tensorflow-probability   0.6.0                


In [11]:
# Zip and download the folder
!zip -r /content/ssd_mobilenet.zip /content/models/fine_tuned/ssd_mobilenet

  adding: content/models/fine_tuned/ssd_mobilenet/ (stored 0%)
  adding: content/models/fine_tuned/ssd_mobilenet/eval_0/ (stored 0%)
  adding: content/models/fine_tuned/ssd_mobilenet/eval_0/events.out.tfevents.1551641957.b79e93b6e78d (deflated 9%)
  adding: content/models/fine_tuned/ssd_mobilenet/events.out.tfevents.1551641296.b79e93b6e78d (deflated 93%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-20000.meta (deflated 93%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-19088.meta (deflated 93%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-18529.index (deflated 72%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-17969.index (deflated 72%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-17969.meta (deflated 93%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-18529.meta (deflated 93%)
  adding: content/models/fine_tuned/ssd_mobilenet/model.ckpt-19648.meta (deflated 93%)
  adding: content/models/fine_tu

# Export the Model
To use the model for inference in production, the graph needs to be freezed.

In [10]:
!python object_detection/export_inference_graph.py \
        --input_type=image_tensor \
        --pipeline_config_path=config/ssd_mobilenet_v1.config \
        --trained_checkpoint_prefix=models/fine_tuned/ssd_mobilenet/model.ckpt-20000 \
        --output_directory=models/exported/ssd_mobilenet


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
117 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-a

In [12]:
# Zip and download the folder
!zip -r /content/ssd_mobilenet_exported.zip /content/models/exported/ssd_mobilenet

  adding: content/models/exported/ssd_mobilenet/ (stored 0%)
  adding: content/models/exported/ssd_mobilenet/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: content/models/exported/ssd_mobilenet/model.ckpt.index (deflated 68%)
  adding: content/models/exported/ssd_mobilenet/saved_model/ (stored 0%)
  adding: content/models/exported/ssd_mobilenet/saved_model/saved_model.pb (deflated 9%)
  adding: content/models/exported/ssd_mobilenet/saved_model/variables/ (stored 0%)
  adding: content/models/exported/ssd_mobilenet/model.ckpt.meta (deflated 93%)
  adding: content/models/exported/ssd_mobilenet/checkpoint (deflated 42%)
  adding: content/models/exported/ssd_mobilenet/frozen_inference_graph.pb (deflated 9%)
  adding: content/models/exported/ssd_mobilenet/pipeline.config (deflated 67%)
